# Object Detection Inference

### Clone the tensorflow models repository if it doesn't already exist

In [1]:
import os
import pathlib

if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 2364, done.
remote: Counting objects: 100% (2364/2364), done.
remote: Compressing objects: 100% (2051/2051), done.
remote: Total 2364 (delta 582), reused 970 (delta 287), pack-reused 0
Receiving objects: 100% (2364/2364), 30.68 MiB | 34.30 MiB/s, done.
Resolving deltas: 100% (582/582), done.


### Install the Object Detection API

In [2]:

%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp36-none-any.whl size=1606870 sha256=6b6c302af71827edf006ee5e4a0abd9cd2458969a69def6c585f0087661dc0a5
  Stored in directory: /tmp/pip-ephem-wheel-cache-h4ipge2r/wheels/94/49/4b/39b051683087a22ef7e80ec52152a27249d1a644ccf4e442ea
  Created wheel for avro-python3: filename=avro_python3-1.10.1-cp36-none-any.whl size=43734 sha256=233b09e1deaf9c7666556d82e340b90fab5f90d98991506db5c5a580143c5def
  Stored in directory: /root/.cache/pip/wheels/65/fe/90/20d6d6d97223d80d20cb390be636619c536edab5658c12bdba
  Created wheel for hdfs: filename=hdfs-2.5.8-cp36-none-any.whl size=33213 sha256=fa50e12c8676779eb57d4e2f305dbc1611c9ad6bcb55aade334f18187b7f46ea
  Stored in directory: /root/.cache/pip/wheels/fe/a7/05/23e3699975fc20f8a30e00ac1e515ab8c61168e982abe4ce70
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=24a896e1c3e89cd0c0c60c4ffb9763d94a8a67ccfca821ea210f51

ERROR: multiprocess 0.70.11.1 has requirement dill>=0.3.3, but you'll have dill 0.3.1.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: apache-beam 2.26.0 has requirement avro-python3!=1.9.2,<1.10.0,>=1.8.1, but you'll have avro-python3 1.10.1 which is incompatible.


### Tensorflow 1

In [3]:
!pip install -U --pre tensorflow-gpu==1.15

     |████████████████████████████████| 411.5MB 43kB/s 
     |████████████████████████████████| 3.8MB 53.0MB/s 
     |████████████████████████████████| 512kB 49.9MB/s 
     |████████████████████████████████| 51kB 8.7MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=b2bbb01b65139d714430a7cc6f378a89eaad9a802347b20b70f0e5d4334ffcce
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.4.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.4.0 has requirement tensorboard~=2.4, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.4.0 has requirement tensorflow-estimator<2.5.0,>=2.4.0rc0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing ins

### Creating Inference Graph

In [56]:
!python /content/drive/MyDrive/Colab_Notebooks/Object_Detection_Training/export_inference_graph.py --input_type=image_tensor --pipeline_config_path=/content/drive/MyDrive/Colab_Notebooks/Object_Detection_Training/faster_rcnn_inception_v2_pets.config --trained_checkpoint_prefix=/content/drive/MyDrive/Colab_Notebooks/Object_Detection_Training/Trained_Model_Checkpoints/model.ckpt-20890 --output_directory=/content/drive/MyDrive/Colab_Notebooks/Object_Detection_Training/inference_graph 


Instructions for updating:
Please use `layer.__call__` method instead.
W0103 18:23:15.495241 140306559809408 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tf_slim/layers/layers.py:2802: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:Scale of 0 disables regularizer.
I0103 18:23:16.728940 140306559809408 regularizers.py:99] Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
I0103 18:23:16.860532 140306559809408 regularizers.py:99] Scale of 0 disables regularizer.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0103 18:23:16.860934 140306559809408 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0103 18:23:16.918875 140306559809408 deprecati

### Now using our Object Detector to Detect Images

In [1]:
!pip install -U --pre tensorflow-gpu==1.15

Requirement already up-to-date: tensorflow-gpu==1.15 in /usr/local/lib/python3.6/dist-packages (1.15.0)


In [2]:
import io
import os
import scipy.misc
import numpy as np
import six
import time
import glob
from IPython.display import display

from six import BytesIO

import matplotlib
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

### Model Preparation

In [9]:
PATH_TO_FROZEN_GRAPH = '/content/drive/MyDrive/Colab_Notebooks/Object_Detection_Training/inference_graph' + '/frozen_inference_graph.pb'
PATH_TO_LABELS = '/content/drive/MyDrive/Colab_Notebooks/Object_Detection_Training/Labelmap.pbtxt'

### Now Load a frozen Tensorflow model into the memory

In [32]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: a file path (this can be local or on colossus)

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

### Loading the LabelMap

In [49]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
category_index = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=3, use_display_name=True)

In [51]:
def run_inference_for_single_image(image, graph):
    if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
    image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

    # Run inference
    output_dict = sess.run(tensor_dict,
                            feed_dict={image_tensor: np.expand_dims(image, 0)})

    # all outputs are float32 numpy arrays, so convert types as appropriate
    output_dict['num_detections'] = int(output_dict['num_detections'][0])
    output_dict['detection_classes'] = output_dict[
        'detection_classes'][0].astype(np.uint8)
    output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
    output_dict['detection_scores'] = output_dict['detection_scores'][0]
    if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict

In [51]:
for image_path in glob.glob('/content/drive/MyDrive/Colab_Notebooks/Object_Detection_Training/Test_Images/ErithacusRubecula0115.jpg'):
  image_np = load_image_into_numpy_array(image_path)
  output_dict = run_inference_for_single_image(PATH_TO_FROZEN_GRAPH,image_np)
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)
  display(Image.fromarray(image_np))